In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import subprocess


In [3]:
class VideoStreamApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Monitor Chicken")

        # Tạo frame cho video
        self.video_frame = tk.Frame(root)
        self.video_frame.pack(pady=10)

        # Nhãn để hiển thị video
        self.video_label = tk.Label(self.video_frame)
        self.video_label.pack()

        # Tạo frame cho ô nhập và nút
        self.control_frame = tk.Frame(root)
        self.control_frame.pack(pady=10)

        # Nhãn và ô nhập địa chỉ RTSP
        self.rtsp_label = tk.Label(self.control_frame, text="Nhập địa chỉ RTSP:")
        self.rtsp_label.pack(side=tk.LEFT)

        self.rtsp_entry = tk.Entry(self.control_frame, width=40)
        self.rtsp_entry.pack(side=tk.LEFT)

        # Nút để bắt đầu phát video
        self.start_button = tk.Button(self.control_frame, text="Bắt đầu phát video", command=self.start_stream)
        self.start_button.pack(side=tk.LEFT)

        # Nút để chuyển đổi sang M3U8
        self.convert_button = tk.Button(self.control_frame, text="Chuyển đổi sang M3U8", command=self.start_conversion)
        self.convert_button.pack(side=tk.LEFT)

        self.vid = None

    def start_stream(self):
        rtsp_url = self.rtsp_entry.get()
        if rtsp_url:
            self.vid = cv2.VideoCapture(rtsp_url)
            if not self.vid.isOpened():
                messagebox.showerror("Error", "Không thể mở video từ URL đã cho.")
            else:
                self.show_frame()
        else:
            messagebox.showwarning("Warning", "Vui lòng nhập URL RTSP.")

    def show_frame(self):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:


                # Thay đổi kích thước khung hình
                resized_frame = cv2.resize(frame, (800, 700))  # Kích thước nhỏ hơn

                # Chuyển đổi khung hình từ BGR sang RGB
                rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

                # Chuyển đổi thành ảnh Pillow
                img = Image.fromarray(rgb_frame)
                imgtk = ImageTk.PhotoImage(image=img)

                # Cập nhật nhãn với ảnh mới
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

                # Gọi lại hàm show_frame để tiếp tục phát video
                self.video_label.after(10, self.show_frame)
            else:
                self.vid.release()
                messagebox.showinfo("Info", "Video đã kết thúc.")

    def start_conversion(self):
        rtsp_url = self.rtsp_entry.get()
        output_file = 'D:/work/Monitor_Chicken/File_M3U8/output.m3u8'  # Tên file đầu ra
        if rtsp_url:
            self.convert_rtsp_to_m3u8(rtsp_url, output_file)
        else:
            messagebox.showwarning("Warning", "Vui lòng nhập URL RTSP.")

    def convert_rtsp_to_m3u8(self, rtsp_url, output_file):
        command = [
            'C:/ffmpeg/ffmpeg.exe',  # Thay đổi đường dẫn tới ffmpeg.exe
            '-i', rtsp_url,
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-f', 'hls',
            output_file
        ]
        
        try:
            subprocess.run(command, check=True)
            messagebox.showinfo("Success", f"Chuyển đổi thành công: {output_file}")
        except subprocess.CalledProcessError as e:
            messagebox.showerror("Error", f"Có lỗi xảy ra: {e}")

    def __del__(self):
        if self.vid is not None:
            self.vid.release()


In [4]:
# Tạo cửa sổ GUI
root = tk.Tk()
app = VideoStreamApp(root)

# Chạy vòng lặp chính
root.mainloop()